In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import pandas as pd
from tqdm import tqdm

# Database connection
engine = create_engine('mssql+pyodbc://Revision-PC\\SQLEXPRESS/RiotDataDB?driver=ODBC+Driver+17+for+SQL+Server')
Session = sessionmaker(bind=engine)

# Function to map user input to team position
def get_team_position():
    position_mapping = {
        '1': 'TOP',
        '2': 'MID',
        '3': 'BOTTOM',
        '4': 'UTILITY',
        '5': 'JUNGLE'
    }
    team_position = input("Select the desired position to be analyzed:\n1 - TOP\n2 - MID\n3 - CARRY\n4 - SUPPORT\n5 - JUNGLE\n")
    return position_mapping.get(team_position, None)

# Function to execute stored procedures
def execute_stored_procedures(session, summoner_name, team_position):
    try:
        # Execute stored procedure for league data
        session.execute(
            text("EXEC sp_GenerateAverageReport_League :teamPosition"),
            {"teamPosition": team_position}
        )
        
        # Execute stored procedure for summoner data
        session.execute(
            text("EXEC sp_GenerateAverageReport :summonerName, :teamPosition"),
            {"summonerName": summoner_name, "teamPosition": team_position}
        )

        # Commit the transaction
        session.commit()
        print("Stored procedures executed successfully.")
    except Exception as e:
        session.rollback()
        print(f"An error occurred during stored procedure execution: {e}")
        raise

# Function to calculate additional metrics
def calculate_advanced_metrics(df):
    df['KDA'] = (df['kills'] + df['assists']) / df['deaths'].replace(0, 1)  # Avoid division by zero
    df['CS_per_Min'] = df['totalMinionsKilled'] / (df['timePlayed'] / 60)
    df['Damage_per_Min'] = df['totalDamageDealtToChampions'] / (df['timePlayed'] / 60)
    df['Vision_per_Min'] = df['visionScore'] / (df['timePlayed'] / 60)
    return df

# Function to fetch and process data from the database
def fetch_and_process_data(engine, summoner_name, team_position):
    try:
        with tqdm(total=2, desc="Querying database") as pbar:
            # Fetch data for summoner
            query_summoner = text("""
                SELECT * 
                FROM dbo.First_Report
                WHERE SummonerName = :summonerName AND PositionPlayed = :teamPosition
            """)
            df_summoner = pd.read_sql(query_summoner, engine, params={"summonerName": summoner_name, "teamPosition": team_position})
            pbar.update(1)

            # Fetch data for league average
            query_league = text("""
                SELECT * 
                FROM dbo.First_Report_Statistics
                WHERE SummonerName = 'League_Average' AND PositionPlayed = :teamPosition
            """)
            df_league = pd.read_sql(query_league, engine, params={"teamPosition": team_position})
            pbar.update(1)

        # Drop unnecessary columns
        columns_to_drop = ['Report_ID', 'SummonerName', 'PositionPlayed', 'ChampionPlayed', 'LastUpdated']
        df_summoner = df_summoner.drop(columns=columns_to_drop, errors='ignore')
        df_league = df_league.drop(columns=columns_to_drop, errors='ignore')

        # Calculate additional metrics
        df_summoner = calculate_advanced_metrics(df_summoner)
        df_league = calculate_advanced_metrics(df_league)

        # Calculate efficiency score
        efficiency_score = (df_summoner.mean(skipna=True) * 100) / df_league.mean(skipna=True)

        # Add overall performance column based on efficiency score
        overall_performance = efficiency_score.apply(lambda x: 'Great' if x > 75 else ('Average' if x > 50 else 'Poor'))

        # Create a final DataFrame for display
        df_final = pd.DataFrame({
            'Summoner_Average': df_summoner.mean(skipna=True),
            'League_Average': df_league.mean(skipna=True),
            'Efficiency_Score': efficiency_score,
            'Overall_Performance': overall_performance
        })

        # Generate visualizations
        generate_visualizations(df_final)

        print(df_final)

    except Exception as e:
        print(f"An error occurred while processing data: {e}")
        raise

# Function to generate visualizations
def generate_visualizations(df_final):
    plt.figure(figsize=(12, 8))

    # Efficiency Score Bar Plot
    sns.barplot(x=df_final.index, y=df_final['Efficiency_Score'])
    plt.xticks(rotation=90)
    plt.title('Efficiency Score by Metric')
    plt.ylabel('Efficiency Score (%)')
    plt.show()

    # Radar Chart for Summoner vs League
    labels = df_final.index
    summoner_values = df_final['Summoner_Average'].values
    league_values = df_final['League_Average'].values

    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    summoner_values = np.concatenate((summoner_values, [summoner_values[0]]))
    league_values = np.concatenate((league_values, [league_values[0]]))
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    ax.fill(angles, summoner_values, color='blue', alpha=0.25)
    ax.fill(angles, league_values, color='red', alpha=0.25)
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, size=10)
    plt.title('Summoner vs League Performance')
    plt.show()

# Main function
def main():
    team_position = get_team_position()
    if not team_position:
        print("Invalid selection. Please select a number between 1 and 5.")
        return

    summoner_name = input("Enter summoner name: ")

    with Session() as session:
        try:
            execute_stored_procedures(session, summoner_name, team_position)
            fetch_and_process_data(engine, summoner_name, team_position)
        except Exception as e:
            print(f"Failed to complete the operation: {e}")

if __name__ == "__main__":
    main()
